In [11]:
import requests
from functools import reduce

In [12]:
import numpy as np
import geopandas as gpd
import pandas as pd

In [13]:
from tqdm.notebook import tqdm

## Open State data

In [15]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

## Get Ethnic Data

In [16]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/groups/B04006.json")
columns_obj = r.json()

In [17]:
columns = []
rename_vars = {}
variables = columns_obj["variables"]
for name, variable in list(variables.items()):
    v_split = variable["label"].split("!!")

    if v_split[0] == "Estimate":
        label = v_split[-1]
        rename_vars[name] = label

    if name.endswith("EA") or name.endswith("MA") or name == "GEO_ID":
        continue
    else:
        columns.append(name)

In [18]:
dfs = []
for i in tqdm(range(0, len(columns), 49), desc="Requesting data in batches"):
    columns_with_geoid = columns[i : i + 49]
    columns_with_geoid.append("GEO_ID")
    columns_formatted = ",".join(columns_with_geoid)
    url = f"https://api.census.gov/data/2023/acs/acs5?get={columns_formatted}&for=county:*"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])

    df = df.drop(columns=["state", "county"], errors="ignore")
    dfs.append(df)

Requesting data in batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [35]:
df = reduce(lambda left, right: pd.merge(left, right, on="GEO_ID", how="inner"), dfs)

In [36]:
df = df.replace(np.nan, 0)

In [37]:
estimate_cols = [col for col in df.columns if col.endswith("E") and col != "NAME"]

In [38]:
formtted_df = df[["GEO_ID", *estimate_cols]]
formtted_df = formtted_df.drop(columns=["NAME"], errors="ignore")

In [39]:
formtted_df[estimate_cols] = formtted_df[estimate_cols].astype(int)

In [40]:
formtted_df["most_common_ancestry_raw"] = formtted_df[
    [
        col
        for col in estimate_cols
        if col not in ["B04006_001E", "B04006_108E", "B04006_109E"]
    ]
].idxmax(axis=1)

/tmp/ipykernel_56202/2831922844.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  formtted_df["most_common_ancestry_raw"] = formtted_df[[col for col in estimate_cols if col not in ['B04006_001E','B04006_108E', 'B04006_109E']]].idxmax(axis=1)


In [41]:
formtted_df["most_common_ancestry_raw"]

0       B04006_036E
1       B04006_005E
2       B04006_005E
3       B04006_005E
4       B04006_036E
           ...     
3217    B04006_005E
3218    B04006_005E
3219    B04006_005E
3220    B04006_005E
3221    B04006_005E
Name: most_common_ancestry_raw, Length: 3222, dtype: object

In [42]:
def check_margin_error(row) -> str:
    geo_id = row["GEO_ID"]
    ethnicity_col = row["most_common_ancestry_raw"]
    val = row[ethnicity_col]

    if not val:
        return None

    moe_col = ethnicity_col.replace("E", "M")
    moe_val = int(df[df["GEO_ID"] == geo_id][moe_col])

    rmoe_val = abs(moe_val / val)
    if rmoe_val < 0.10:
        return variables[ethnicity_col]["label"].split("!!")[-1]
    else:
        return None

In [43]:
formtted_df["most_common_ancestry"] = formtted_df.apply(
    lambda row: check_margin_error(row), axis=1
)

/tmp/ipykernel_56202/1256534131.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  moe_val = int(df[df["GEO_ID"] == geo_id][moe_col])
/tmp/ipykernel_56202/2401641906.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  formtted_df["most_common_ancestry"] = formtted_df.apply(


In [44]:
rename_vars["GEO_ID"] = "GEOIDFQ"
formtted_df = formtted_df.rename(columns=rename_vars)

In [46]:
formtted_df.groupby("most_common_ancestry").size().reset_index(
    name="COUNT"
).sort_values("COUNT", ascending=False)

,most_common_ancestry,COUNT
5,German,908
2,English,183
6,Irish,69
0,American,57
7,Italian,28
4,French (except Basque),11
8,Norwegian,8
12,West Indian (except Hispanic groups):,6
1,Dutch,3
3,Finnish,2


In [50]:
formtted_df[formtted_df["most_common_ancestry"] == "Subsaharan African:"]

,GEOIDFQ,Jordanian,Iraqi,Egyptian,Total:,Arab:,American,Alsatian,Albanian,Afghan,...,West Indian,U.S. Virgin Islander,Trinidadian and Tobagonian,Unclassified or not reported,Other groups,Jamaican,Haitian,Dutch West Indian,most_common_ancestry_raw,most_common_ancestry
1210,0500000US24033,182,68,1454,955584,4041,26811,28,223,1747,...,3214,587,3707,188886,571799,16603,5598,8,B04006_073E,Subsaharan African:


## Merge Data

In [47]:
gdf = counties_gdf.merge(formtted_df, on="GEOIDFQ", how="inner")

In [48]:
gdf = gdf.to_crs(9311)
gdf.to_file("data/Largest_Ancestry_Per_County.gpkg")